# Empirical aDDM Inference

This notebook will simulate and recover data using `efficient-fpt`'s pipeline.

### Simulating data

In [1]:
import os, sys

DDM_dir = os.path.abspath('/Users/braydenchien/Desktop/Enkavilab/DDM')
sys.path.append(DDM_dir)

dt = 0.001

In [2]:
from simulation import get_corrected_empirical_distributions
from ast import literal_eval
import numpy as np
import pandas as pd

df_raw = pd.read_csv('/Users/braydenchien/Desktop/Enkavilab/DDM/1ms_trial_data.csv')
df_raw['choice'] = df_raw['choice'].map({'left':0,'right':1})
df_raw['RT'] = df_raw['RT'] / dt # adjustment for RT
df_raw['fixation'] = df_raw['fixation'].apply(literal_eval)

to_drop = pd.read_csv("/Users/braydenchien/Desktop/Enkavilab/DDM/dropped_trials.csv").rename(columns={"parcode": "sub_id"})

df = df_raw[
    ~df_raw.set_index(["sub_id", "trial"]).index.isin(
        to_drop.set_index(["sub_id", "trial"]).index
    )
]

num_data, _ = df.shape

value_diffs = np.unique(df['avgWTP_left'] - df['avgWTP_right'])
legend = {
    "left": {1},
    "right": {2},
    "transition": {0}, 
    "blank_fixation": {4}
}
fixation_col = 'fixation'
left_value_col = 'avgWTP_left'
right_value_col = 'avgWTP_right'

empirical_distributions = get_corrected_empirical_distributions(
    df,
    value_diffs=value_diffs,
    legend=legend,
    fixation_col=fixation_col,
    left_value_col=left_value_col,
    right_value_col=right_value_col,
    cutoff=0.9
)

In [3]:
# Defining constants
a, b = 1, 0
eta = 0.5
kappa = 2
T = 25 / dt # Bounded by Markov w/p 0.9, E[T] = 2.5, in this case, in ms
x0 = 0
sigma = 0.7 # As calculated from Eum et al. (2023)

# r1_data = np.zeros(num_data)
# r2_data = np.zeros(num_data)
r1_data = df['avgWTP_left'].to_numpy()
r2_data = df['avgWTP_right'].to_numpy()
# r1_data = [3]*num_data
# r2_data = [3]*num_data

seeds = np.random.SeedSequence(123).spawn(num_data)

In [4]:
from joblib import Parallel, delayed, cpu_count, dump, load
from data_utils import simulate_empirical_trial
import time

print("Available jobs:", cpu_count())

start = time.time()

dump(empirical_distributions, "empirical.mmap")
empirical_distributions = load("empirical.mmap", mmap_mode="r")

results = Parallel(n_jobs=-1, backend="loky")(
    delayed(simulate_empirical_trial)(
        n,
        r1_data,
        r2_data,
        empirical_distributions,
        eta=eta,
        kappa=kappa,
        sigma=sigma,
        a=a,
        b=b,
        T=T,
        x0=x0,
        dt=dt,
        seed=seeds[n]
    )
    for n in range(num_data)
)

print(f"Elapsed time: {time.time() - start:.3f} seconds")

# Store results
decision_data = np.zeros((num_data, 2))
flag_data = np.zeros(num_data)
mu_data = [None] * num_data
sacc_data = [None] * num_data

for n, (decision, mu_array, sacc_array, r1, r2, flag) in enumerate(results):
    decision_data[n] = decision
    mu_data[n] = mu_array
    sacc_data[n] = sacc_array
    r1_data[n] = r1
    r2_data[n] = r2
    flag_data[n] = flag

Available jobs: 8
Elapsed time: 214.204 seconds


In [5]:
d_data = np.array([len(mu_array) for mu_array in mu_data])
max_len = np.max(d_data)

mu_data_padded = np.array([np.pad(mu_array, (0, max_len - len(mu_array)), mode='constant') for mu_array in mu_data])
sacc_data_padded = np.array([np.pad(sacc_array, (0, max_len - len(sacc_array)), mode='constant') for sacc_array in sacc_data])

The cell below is a check to see if all the data needed for inference is generated. The source notebook for addm inference unpacks the same data, so this is a sanity check.

In [6]:
# data_to_save = {
#     'mu_array_padded_data': mu_data_padded, # Drift rates for each stage
#     'sacc_array_padded_data': sacc_data_padded, # Saccade lengths for each stage
#     'd_data': d_data, # Number of stages
#     'decision_data': decision_data, # RT and choice
#     'r1_data': r1_data, # avgWTP_left
#     'r2_data': r2_data, # avgWTP_right
#     'flag_data': flag_data.astype(np.int32), # Left choice first
#     'eta': eta, # theta (multiplicative discount factor)
#     'kappa': kappa, # drift
#     'sigma': sigma, # noise
#     'a': a, # initial boundary height
#     'b': b, # linear boundary collapse rate
#     'x0': x0, # initial diffusion distribution
#     'T': T, # Maximum simulation horizon
# }

### Recovering data

In [7]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import pickle
import time
import seaborn as sns
from scipy.optimize import fmin, minimize, LinearConstraint, Bounds

from efficient_fpt.multi_stage_cy import compute_loss_parallel, print_num_threads

In [8]:
DATA_TYPE = np.float64

# a = data["a"]
# b = data["b"]
# x0 = data["x0"]
# mu1_true = data["mu1"] # Deprecated
# mu2_true = data["mu2"] # Deprecated
eta_true = eta
kappa_true = kappa
# r1_data = data["r1_data"]
# r2_data = data["r2_data"]
flag_data = flag_data.astype(np.int32)

# sigma = data["sigma"]
# T = data["T"]

mu1_true_data = kappa_true * (r1_data - eta_true * r2_data)
mu2_true_data = kappa_true * (eta_true * r1_data - r2_data)

mu_true_data = mu_data_padded.astype(DATA_TYPE)
sacc_data = sacc_data_padded.astype(DATA_TYPE)
length_data = d_data.astype(np.int32)
rt_data = decision_data[:, 0].astype(DATA_TYPE)
choice_data = decision_data[:, 1].astype(np.int32)

num_data, max_d = mu_true_data.shape

In [9]:
print_num_threads()
print("# data =", num_data)

Number of available threads: 8
# data = 18758


In [10]:
num_iter = 10
start_time = time.time()
for _ in range(num_iter):
    loss = compute_loss_parallel(
        mu1_true_data,
        mu2_true_data,
        rt_data,
        choice_data,
        flag_data,
        sacc_data,
        length_data,
        max_d,
        sigma,
        a,
        b,
        x0,
        num_threads=10,
    )
end_time = time.time()
print(f"Likelihood evaluation time: {(end_time - start_time) / num_iter:.3f} s")

Likelihood evaluation time: 0.168 s


In [11]:
# ============================================================
# Constraint optimization for searching all parameters
# ============================================================

print("\nNumerical optimization for eta, kappa, a, b, x0:")
method = "trust-constr"
print("Using " + method)

# ------------------------------------------------------------
# ORIGINAL 5-PARAMETER OBJECTIVE (COMMENTED OUT)
# ------------------------------------------------------------
# func = lambda paras: compute_loss_parallel(
#     paras[1] * (r1_data - paras[0] * r2_data),
#     paras[1] * (paras[0] * r1_data - r2_data),
#     rt_data,
#     choice_data,
#     flag_data,
#     sacc_data,
#     length_data,
#     max_d,
#     sigma,
#     paras[2],
#     paras[3],
#     paras[4],
# )

# ------------------------------------------------------------
# UPDATED OBJECTIVE: REDUCED PARAMETER SPACE
# ------------------------------------------------------------
# We now optimize only [eta, kappa, a].
# b and x0 are FIXED to 0 to improve identifiability.
#
# Rationale:
# - Removes collapsing boundary degeneracy (b)
# - Removes starting bias compensation (x0)
# - Forces asymmetry to be explained via drift
# - Improves conditioning of parameter recovery
#
# Parameter vector:
# paras = [eta, kappa, a]

func = lambda paras: compute_loss_parallel(
    paras[1] * (r1_data - paras[0] * r2_data),
    paras[1] * (paras[0] * r1_data - r2_data),
    rt_data,
    choice_data,
    flag_data,
    sacc_data,
    length_data,
    max_d,
    sigma,
    paras[2],  # a
    0.0,       # b fixed to 0 (no collapsing bounds)
    0.0,       # x0 fixed to 0 (no starting bias)
)

# ------------------------------------------------------------
# BOUNDS
# ------------------------------------------------------------
# ORIGINAL 5-PARAMETER BOUNDS (COMMENTED OUT)
# [eta, kappa, a, b, x0]
#
# bounds = Bounds(
#     [0, 0, 0, 0, -np.inf],
#     [1, np.inf, np.inf, np.inf, np.inf]
# )

# ------------------------------------------------------------
# ORIGINAL "LOCKED" VERSION WITH REDUNDANT DIMENSIONS (COMMENTED OUT)
# These bounds forced b = 0 and x0 = 0, but still optimized
# over 5 dimensions and required unnecessary constraints.
#
# bounds = Bounds(
#     [0, 0, 0, 0, 0],
#     [1, np.inf, np.inf, 0, 0]
# )

# ------------------------------------------------------------
# UPDATED 3-PARAMETER BOUNDS
# ------------------------------------------------------------
# Only optimize over [eta, kappa, a]
# All constraints on b and x0 are removed because those
# parameters are no longer optimized.
#
# eta ∈ [0,1]
# kappa ≥ 0
# a ≥ 0

bounds = Bounds(
    [0, 0, 0],
    [1, np.inf, np.inf],
)

# ------------------------------------------------------------
# LINEAR CONSTRAINTS
# ------------------------------------------------------------
# ORIGINAL CONSTRAINTS (COMMENTED OUT)
#
# These enforced:
# 1) a - max(rt)*b ≥ 0  (collapsing boundary validity)
# 2) |x0| ≤ a          (initial state inside bounds)
#
# con = LinearConstraint(
#     [[0, 0, 1, -np.max(rt_data), 0],
#      [0, 0, 1, 0, 1],
#      [0, 0, 1, 0, -1]],
#     lb=[0, 0, 0],
#     ub=[np.inf, np.inf, np.inf],
# )

# ------------------------------------------------------------
# UPDATED: NO LINEAR CONSTRAINTS NEEDED
# ------------------------------------------------------------
# Since b = 0 and x0 = 0 are fixed:
# - Boundary collapse constraint is irrelevant
# - Starting point constraint is automatically satisfied
# - a ≥ 0 is already enforced by bounds
#
# Therefore we REMOVE LinearConstraint entirely.

# ------------------------------------------------------------
# INITIAL GUESS
# ------------------------------------------------------------
# ORIGINAL 5-PARAMETER GUESS (COMMENTED OUT)
# initial_guess = [0.5, 1, 1.0, 0.0, 0.0]

# Updated 3-parameter guess
initial_guess = [0.5, 1, 1.0]

print("Initial guess:", initial_guess)
print()

# ------------------------------------------------------------
# OPTIMIZATION
# ------------------------------------------------------------
start_time = time.time()

paras_opt_result = minimize(
    func,
    x0=initial_guess,
    bounds=bounds,
    method=method,
    options={"verbose": 1},
)

print(f"Total time: {time.time() - start_time:.3f} seconds")
print(paras_opt_result)


Numerical optimization for eta, kappa, a, b, x0:
Using trust-constr
Initial guess: [0.5, 1, 1.0]

`gtol` termination condition is satisfied.
Number of iterations: 33, function evaluations: 92, CG iterations: 61, optimality: 4.86e-09, constraint violation: 0.00e+00, execution time: 1.7e+01 s.
Total time: 17.342 seconds
           message: `gtol` termination condition is satisfied.
           success: True
            status: 1
               fun: 0.6742021889224282
                 x: [ 9.991e-01  5.714e-01  7.220e-01]
               nit: 33
              nfev: 92
              njev: 23
              nhev: 0
          cg_niter: 61
      cg_stop_cond: 1
              grad: [-1.553e-05  1.490e-08  0.000e+00]
   lagrangian_grad: [-2.968e-12 -7.432e-10 -4.860e-09]
            constr: [array([ 9.991e-01,  5.714e-01,  7.220e-01])]
               jac: [<Compressed Sparse Row sparse matrix of dtype 'float64'
                    	with 3 stored elements and shape (3, 3)>]
       constr_nfev: [0]

In [12]:
mle = paras_opt_result["x"]
print("True and estimated value of parameters:")
print(f"eta: {eta_true:.5f}, {mle[0]:.5f}")
print(f"kappa: {kappa_true:.5f}, {mle[1]:.5f}")
print(f"a: {a:.5f}, {mle[2]:.5f}")
# print(f"b: {b:.5f}, {mle[3]:.5f}")
# print(f"x0: {x0:.5f}, {mle[4]:.5f}")

True and estimated value of parameters:
eta: 0.50000, 0.99912
kappa: 2.00000, 0.57142
a: 1.00000, 0.72199


Without proceeding onto further parameter recovery methods, I believe that having two methods (differential evolution and maximum likelihood estimation) converging on different parameters suggests that the data simulation process does not generate data identifiable to synthetic parameters. Specifically with this model, there is flexibility in collapsing bounds and initial condition, so it does not fit the class of model that I am using. From this, I have three thoughts. First, is it possible to map the transformation of values by carefully selecting different test points? Second, can I constrain the parameter space, like setting x0 to 0 or b to 0? Third, if the crux of the issue is truly the empirical simulation methodology, can some of the mystery be explained by finding out why middle fixations have value-modulated effects if they are sampled iid?